In [2]:
import pandas as pd
import json

# 读取Excel文件
excel_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data/cleaned_comments_dianping_hotpot.xlsx'  # 替换为你的Excel文件路径
sheet_name = 'Sheet1'  # 替换为你的Sheet名称

# 读取指定列
df = pd.read_excel(excel_path, sheet_name=sheet_name, usecols=['cleaned_comment'])

# 将列数据转换为列表，并处理datetime对象
def convert_to_serializable(val):
    if isinstance(val, pd.Timestamp):
        return val.strftime('%Y-%m-%d')
    return str(val)  # 将所有非字符串类型转换为字符串

title_4_list = df['cleaned_comment'].dropna().apply(convert_to_serializable).tolist()

# 保存为JSON文件
json_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data/cleaned_comments_dianping_hotpot.json'  # 替换为你的输出JSON文件路径
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(title_4_list, f, ensure_ascii=False, indent=4)

print(f"数据已保存到 {json_path}！")


数据已保存到 /data1/dxw_data/llm/RAG-mkt-kmeans/data/cleaned_comments_dianping_hotpot.json！


In [2]:
import json
import numpy as np
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

# Load the JSON file
json_file_path = "/data1/dxw_data/llm/RAG-mkt-kmeans/data/cleaned_comments_dianping_hotpot.json"
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Load the pre-trained model
model_path = '/data1/dxw_data/llm/text2vec-large-chinese'
model = SentenceTransformer(model_path)

# Convert each element in the JSON file to a vector
vectors = model.encode(data)

# Perform KMeans clustering
num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(vectors)

# Get cluster labels
labels = kmeans.labels_

# Save clustering results
results = {"text": data, "cluster": labels.tolist()}
output_file_path = "/data1/dxw_data/llm/RAG-mkt-kmeans/data/clustering_results.json"
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# Find the representative text for each cluster
representatives = []
for i in range(num_clusters):
    cluster_indices = np.where(labels == i)[0]
    cluster_vectors = vectors[cluster_indices]
    cluster_center = kmeans.cluster_centers_[i]
    distances = np.linalg.norm(cluster_vectors - cluster_center, axis=1)
    representative_index = cluster_indices[np.argmin(distances)]
    representatives.append(data[representative_index])

# Save the representative texts
representative_file_path = "/data1/dxw_data/llm/RAG-mkt-kmeans/data/representative_texts.json"
with open(representative_file_path, 'w', encoding='utf-8') as f:
    json.dump(representatives, f, ensure_ascii=False, indent=4)

print("Clustering and representative text extraction complete. Results saved to:")
print("Clustering results:", output_file_path)
print("Representative texts:", representative_file_path)


No sentence-transformers model found with name /data1/dxw_data/llm/text2vec-large-chinese. Creating a new one with MEAN pooling.
/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clustering and representative text extraction complete. Results saved to:
Clustering results: /data1/dxw_data/llm/RAG-mkt-kmeans/data/clustering_results.json
Representative texts: /data1/dxw_data/llm/RAG-mkt-kmeans/data/representative_texts.json
